# Introduction

Problems from leetcode

### Autocomplete

In [60]:
from heapq import heappop, heappush
from collections import defaultdict

class AutocompleteSystem:

    def __init__(self, sentences, times):
        self.cache = defaultdict(int)
        for s,t in zip(sentences, times):
            self.cache[s] = t
            self.past_prefix = ''

    def inp(self, c):
        """
        
        BRUTE FORCE:
        - select top k from list that match c
        
        Search: Time: O(n*logk), Space: O(k)
        Insert: Time: O(1)
        
        Ex: s = 'i love'
        
        BETTER:
        - use an ordered dict?
        - that way I can stop after the first three hits?
        - (but are ordered dicts just a hash?)
        
        - searching ordered dict --> O(n)  
        - inserting to dict -->  O(log n)
        
        
        TO DO / CONSIDER:
        - equal frequency sentences
        - no matches 
        
        """
                
        #Do the autocompletion
        if c != '#':
        
            prefix = self.past_prefix + c
            self.past_prefix = prefix

            #Return top 3 candidates that match the prompt
            l = len(prefix)
            heap = []
            for sentence, time in self.cache.items():
                if sentence[:l] == prefix:
                    if len(heap) < 3:
                        heappush(heap,(time,sentence))
                    else:
                        #keep equal matches
                        if time == heap[0][0]:
                            heappush(heap,(time,sentence))
                        elif time > heap[0][0]: 
                            heappop(heap)
                            heappush(heap,(time,sentence))
            return heap

        #Add new sentence to cache
        else:
            new_sentence = self.past_prefix
            self.cache[new_sentence] += 1
            self.past_prefix = ''
            return []

#Set up
sentences = ['i love you', 'island','ironman','i love leetcode']
times = [5,3,2,2]
obj = AutocompleteSystem(sentences, times)

#Start the prompt
obj.inp('i')
#obj.inp(' ')
#obj.inp('a')
#obj.inp('#')

[(2, 'i love leetcode'), (2, 'ironman'), (3, 'island'), (5, 'i love you')]

In [59]:
heap = [(2, 'i love leetcode'), (2, 'ironman'), (3, 'island'), (5, 'i love you')]
heap

[(2, 'i love leetcode'), (2, 'ironman'), (3, 'island'), (5, 'i love you')]

In [38]:
temp = obj.cache
sorted(temp.keys())

['i a', 'i love leetcode', 'i love you', 'ironman', 'island']

In [40]:
temp['i a'] += 7

NameError: name 'tmep' is not defined

In [61]:
for k,v in temp.items():
    print('k,v = {},{}'.format(k,v))

k,v = i love you,5
k,v = island,3
k,v = ironman,2
k,v = i love leetcode,2
k,v = i a,8


### Alien dictionary

In [80]:
from collections import defaultdict

def f(s):
    """
    
    Ex:
    
    s = ["wrt","wrf","er", "ett", "rftt"]
    out = wertf
    
    IDEAS:
    - topological sort?
    - draw graph
    - then do topological sort
    - if there is no topological sort, then return ''
    
    Time: O(V + E) = O(n + m), where n = len(s), m = max len(s_i)
    Space: O(n + m)
        
    But expect Time to be smaller in practice    
    
    """
    
    G = make_graph(s)
    result = topological_sort(G)
    return result if result else ''

def make_graph(s):
    G = defaultdict(list)
    for word in s:
        if len(word) == 1:
            G[word[0]] = []
        else:
            for i in range(len(word)-1):
                curr,nxt = word[i], word[i+1]
                if nxt not in G[curr]:
                    G[curr].append(nxt)
    return G

def topological_sort(G):
    
    result = []
    visited = set()
    
    def helper(G,node):
        visited.add(node)
        for n in G[node]:
            if n not in visited:
                helper(G,n)
        result.insert(0,node)  #put parent nodes at front
        
    for node in G.keys():
        if node not in visited:
            helper(G,node)
    return result


s = ["wrt","wrf","er", "ett", "rftt"]
s = ['z','x','z']
G = make_graph(s)
topological_sort(G)

['x', 'z']

In [90]:
G = {'a':['b'], 'b':['c'], 'c':['d'],'d':[]}
topological_sort(G)

['a', 'b', 'c', 'd']

In [91]:
def detect_cycle(G,start):
    visited = set()
    stack = [(start,[start])]
    while stack:
        node, path = stack.pop(-1)
        visited.add(node)
        print('path = {}'.format(path))
        for n in G[node]:
            if n in path:
                return True
            if n not in visited:                
                stack.append((n,path+[n]))
    return False

G = {'a':['b'], 'b':['c'], 'c':['d'],'d':[]}
detect_cycle(G,'a')

path = ['a']
path = ['a', 'b']
path = ['a', 'b', 'c']
path = ['a', 'b', 'c', 'd']


False

### Decode ways

In [146]:
from functools import lru_cache

#@lru_cache(maxsize=None)
def f(s):
    """
    
    BRUTE FORCE:
    - enumerate all 'permutations'
    - count allowable ones
    
    Time: O(n!), Space: O(1)
    
    BETTER:
    - recursion with caching
    
    Time: O(n), Space: O(n)
    
    """
            
    if len(s) == 0:
        yield 'flag'       
    if len(s) == 1:
        yield [s]
    else:
        for i in range(1,len(s)+1):  # s = 'abc'. first: 'a, bc', last: '', 'abc'
            left, right = s[:i], s[i:]
            for p in f(right):
                if p != 'flag':
                    yield [left] + p
                else:
                    yield [left]  
                    
s = 'abc'
list(f(s))

[['a', 'b', 'c'], ['a', 'bc'], ['ab', 'c'], ['abc']]

In [190]:
def dp(s):
    """
    
    Simpler case: pretend there is map for all two digits numbers: 1->A, ... 99->Z!
    
    P[i] = number of ways s_1, ... s_{i} can be sorted
        
    Boundary conditions
    P[0] = 0
    P[1] = 1
    P[2] = 2
    P[3] = ?
    
    Ex: s = '1215'
    
    P[1] = [1]
    P[2] = [[1,2],[12]]
    
    P[3] = [[1,2,1],[12,1]]
    P[4] = [[1,2,1,5],[12,1,5], |||, [1,2,15], [12,15] ]  
    
    - third number can only pair with neighbour?
    
    if 0 < s[i] <= 0:
        P[i] = P[i-1]
    if 0 < s[i-1:i+1] <= 26:
        P[i] = P[i-2] + 1 ?
        
    """
    
    #edge case
    if s[0] == '0':a
        return 0
    if len(s) == 0:
        return 0
    
    #BCs
    P = [0]*(len(s)+1)
    P[0] = 1 
    
    for i in range(1,len(s)+1):
        j = i-1
        
        #single digits
        if 0 < int(s[j]) <= 9:
            P[i] = P[i-1]
        
        #double digits
        #print('i, s[i-1:i+1] = {},{}'.format(i, s[j-1:j+1]))
        if s[j-1:j+1] and 10 <= int(s[j-1:j+1]) <= 26:
            P[i] += P[i-2]
            
    return P

s = '1'
dp(s)

i, s[i-1:i+1] = 1,1


[1, 1]